In [1]:
#!/usr/bin/env python
"""Convolutional variational auto-encoder for binarized MNIST.

The neural networks are written with TensorFlow Slim.

References
----------
http://edwardlib.org/tutorials/decoder
http://edwardlib.org/tutorials/inference-networks
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import os
import tensorflow as tf

from edward.models import Bernoulli, Normal
from edward.util import Progbar
from observations import mnist
from scipy.misc import imsave
from tensorflow.contrib import slim

In [2]:
def generator(array, batch_size):
  """Generate batch with respect to array's first axis."""
  start = 0  # pointer to where we are in iteration
  while True:
    stop = start + batch_size
    diff = stop - array.shape[0]
    if diff <= 0:
      batch = array[start:stop]
      start += batch_size
    else:
      batch = np.concatenate((array[start:], array[:diff]))
      start = diff
    batch = batch.astype(np.float32) / 255.0  # normalize pixel intensities
    batch = np.random.binomial(1, batch)  # binarize images
    yield batch


def generative_network(z):
  """Generative network to parameterize generative model. It takes
  latent variables as input and outputs the likelihood parameters.

  logits = neural_network(z)
  """
  with slim.arg_scope([slim.conv2d_transpose],
                      activation_fn=tf.nn.elu,
                      normalizer_fn=slim.batch_norm,
                      normalizer_params={'scale': True}):
    net = tf.reshape(z, [M, 1, 1, d])
    net = slim.conv2d_transpose(net, 128, 3, padding='VALID')
    net = slim.conv2d_transpose(net, 64, 5, padding='VALID')
    net = slim.conv2d_transpose(net, 32, 5, stride=2)
    net = slim.conv2d_transpose(net, 1, 5, stride=2, activation_fn=None)
    net = slim.flatten(net)
    return net


def inference_network(x):
  """Inference network to parameterize variational model. It takes
  data as input and outputs the variational parameters.

  loc, scale = neural_network(x)
  """
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.elu,
                      normalizer_fn=slim.batch_norm,
                      normalizer_params={'scale': True}):
    net = tf.reshape(x, [M, 28, 28, 1])
    net = slim.conv2d(net, 32, 5, stride=2)
    net = slim.conv2d(net, 64, 5, stride=2)
    net = slim.conv2d(net, 128, 5, padding='VALID')
    net = slim.dropout(net, 0.9)
    net = slim.flatten(net)
    params = slim.fully_connected(net, d * 2, activation_fn=None)

  loc = params[:, :d]
  scale = tf.nn.softplus(params[:, d:])
  return loc, scale


In [3]:
ed.set_seed(42)

data_dir = "/tmp/data"
out_dir = "/tmp/out"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
M = 128  # batch size during training
d = 10  # latent dimension

# DATA. MNIST batches are fed at training time.
(x_train, _), (x_test, _) = mnist(data_dir)
x_train_generator = generator(x_train, M)

# MODEL
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
logits = generative_network(z)
x = Bernoulli(logits=logits)

# INFERENCE
x_ph = tf.placeholder(tf.int32, [M, 28 * 28])
loc, scale = inference_network(tf.cast(x_ph, tf.float32))
qz = Normal(loc=loc, scale=scale)

# Bind p(x, z) and q(z | x) to the same placeholder for x.
data = {x: x_ph}
inference = ed.KLqp({z: qz}, data)
optimizer = tf.train.AdamOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)

hidden_rep = tf.sigmoid(logits)


>> Downloading /tmp/data/train-images-idx3-ubyte.gz.part 
>> [9.5 MB/9.5 MB] 105% @2.1 MB/s,[0s remaining, 4s elapsed]          
URL https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz downloaded to /tmp/data/train-images-idx3-ubyte.gz 
>> Downloading /tmp/data/train-labels-idx1-ubyte.gz.part 
>> [28.2 KB/28.2 KB] 3630% @5.1 MB/s,[0s remaining, 0s elapsed]        
URL https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz downloaded to /tmp/data/train-labels-idx1-ubyte.gz 
>> Downloading /tmp/data/t10k-images-idx3-ubyte.gz.part 
>> [1.6 MB/1.6 MB] 127% @3.0 MB/s,[0s remaining, 0s elapsed]        
URL https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz downloaded to /tmp/data/t10k-images-idx3-ubyte.gz 
>> Downloading /tmp/data/t10k-labels-idx1-ubyte.gz.part 
>> [4.4 KB/4.4 KB] 23086% @6.6 MB/s,[0s remaining, 0s elapsed]        
URL https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz downlo

In [4]:
tf.global_variables_initializer().run()

n_epoch = 100
n_iter_per_epoch = x_train.shape[0] // M
for epoch in range(1, n_epoch + 1):
  print("Epoch: {0}".format(epoch))
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)
    x_batch = next(x_train_generator)
    info_dict = inference.update(feed_dict={x_ph: x_batch})
    avg_loss += info_dict['loss']

  # Print a lower bound to the average marginal likelihood for an
  # image.
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / M
  print("-log p(x) <= {:0.3f}".format(avg_loss))

  # Visualize hidden representations.
  images = hidden_rep.eval()
  for m in range(M):
    imsave(os.path.join(out_dir, '%d.png') % m, images[m].reshape(28, 28))


Epoch: 1
468/468 [100%] ██████████████████████████████ Elapsed: 110s
-log p(x) <= 216.446
Epoch: 2
  1/468 [  0%]                                ETA: 0s

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:25: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


468/468 [100%] ██████████████████████████████ Elapsed: 111s
-log p(x) <= 132.551
Epoch: 3
468/468 [100%] ██████████████████████████████ Elapsed: 109s
-log p(x) <= 121.977
Epoch: 4
468/468 [100%] ██████████████████████████████ Elapsed: 109s
-log p(x) <= 117.383
Epoch: 5
468/468 [100%] ██████████████████████████████ Elapsed: 106s
-log p(x) <= 114.721
Epoch: 6
468/468 [100%] ██████████████████████████████ Elapsed: 104s
-log p(x) <= 112.887
Epoch: 7
468/468 [100%] ██████████████████████████████ Elapsed: 105s
-log p(x) <= 111.487
Epoch: 8
468/468 [100%] ██████████████████████████████ Elapsed: 105s
-log p(x) <= 110.465
Epoch: 9
468/468 [100%] ██████████████████████████████ Elapsed: 106s
-log p(x) <= 109.619
Epoch: 10
468/468 [100%] ██████████████████████████████ Elapsed: 106s
-log p(x) <= 108.966
Epoch: 11
468/468 [100%] ██████████████████████████████ Elapsed: 108s
-log p(x) <= 108.320
Epoch: 12
468/468 [100%] ██████████████████████████████ Elapsed: 107s
-log p(x) <= 107.658
Epoch: 13
468/46

KeyboardInterrupt: 